In [2]:
!which python

/mnt/mydisk/.pyenv/versions/3.8.16/envs/crawl-data-for-sale/bin/python


In [4]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 2.3 MB/s eta 0:00:00m eta 0:00:010:00:010m
  Using cached certifi-2023.7.22-py3-none-any.whl (158 kB)
  Using cached urllib3-2.0.4-py3-none-any.whl (123 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 KB 4.3 MB/s eta 0:00:004.5 MB/s eta 0:00:01
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached idna-3.4-py3-none-any.whl (61 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
You should consider upgrading via the '/mnt/mydisk/.pyenv/versions/3.8.16/envs/crawl-data-for-sale/bin/python3.8 -m pip install --upgrade pip' command.


In [7]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
  Running setup.py install for bs4 ... done
You should consider upgrading via the '/mnt/mydisk/.pyenv/versions/3.8.16/envs/crawl-data-for-sale/bin/python3.8 -m pip install --upgrade pip' command.


In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [89]:
import csv
import os

def append_to_csv(file_path, data, header):
    file_exists = os.path.exists(file_path)

    # Open the CSV file in append mode
    with open(file_path, mode='a', newline='', encoding='utf-8') as csv_file:
        # Create a CSV writer
        csv_writer = csv.writer(csv_file)

        # If the file is new, write the header
        if not file_exists:
            csv_writer.writerow(header)

        # Split the data string into individual values
        values = data.split(', ')

        # Write the values to the CSV file
        csv_writer.writerow(values)

# Example usage:



In [72]:
import time

# Function to scroll to the end of the page and click the "Tiếp" link
def scroll_to_end_and_click_next(driver):
    while True:
        # Scroll to the end of the page
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for a short time to load more content if necessary (adjust this as needed)
        time.sleep(2)

        # Find the "Tiếp" (Next) link
        next_link = driver.find_element_by_link_text('Tiếp')

        # If the "Tiếp" link is not displayed, break the loop (reached the last page)
        if not next_link.is_displayed():
            break

        # Click the "Tiếp" link
        next_link.click()

In [157]:
from bs4 import BeautifulSoup
import re

def extractInfo(soup):
    try:
        nganh_listing_txt = ''
        logo_url = ''
        company_name = ''
        description = ''
        address = ''
        email = ''
        website = ''
        phone_numbers = []
        district = ''
        is_verified = 'yes'

        try:
            logo_url = soup.find('div', class_='logo_congty').find('img')['src']
        except:
            pass

        # Extract company name
        try:
            company_name_elem = soup.find('div', class_='listings_center')
            if company_name_elem is not None:
                company_name_elem = company_name_elem.find('a')
                company_name = company_name_elem.text.strip() if company_name_elem else ''
            else:
                company_name_elem = soup.find('div', class_='listings_center_khongxacthuc')
                company_name_elem = company_name_elem.find('a')
                if company_name_elem:
                    company_name = company_name_elem.text.strip() if company_name_elem else ''
                    is_verified = 'no'
        except:
            pass

        # Find the <div> with class "logo_congty_diachi"
        try:
            logo_congty_diachi_div = soup.find('div', class_=['logo_congty_diachi', 'listing_diachi_nologo'])

            # Extract nganh_listing_txt
            nganh_listing_txt_elem = logo_congty_diachi_div.find('span', class_='nganh_listing_txt')
            nganh_listing_txt = nganh_listing_txt_elem.text.strip() if nganh_listing_txt_elem else ''

            # Extract the address
            address_elem = logo_congty_diachi_div.find('small')
            address = address_elem.text.strip() if address_elem else ''
            # Extract the phone numbers
            phone_numbers = []
            listing_dienthoai_div = logo_congty_diachi_div.find('div', class_='listing_dienthoai')
            if listing_dienthoai_div:
                phone_links = listing_dienthoai_div.find_all('a', href=True)
                for phone_link in phone_links:
                    hef = phone_link.get('href', '').strip()
                    if hef.startswith('tel:'):
                        phone_numbers.append(hef.replace('tel:', ''))
            else:
                phone_link = logo_congty_diachi_div.find('div', class_='p-2 pt-0 ps-0 pe-4 pb-0').find('a')
                hef = phone_link.get('href', '').strip()
                if hef.startswith('tel:'):
                    phone_numbers.append(hef.replace('tel:', ''))
        
            # Define a regular expression pattern to match "Q. [District]" before "Tp. Hồ Chí Minh"
            pattern = r'(.+?),\s*Tp\. Hồ Chí Minh'
            # Use regular expression to find the matching part in the address
            match = re.search(pattern, address)
            # Extract the matching part if found
            if match:
                extracted_text = match.group(1).strip()
                district = extracted_text.split(', ')[-1]
            else:
                extracted_text = None
        except:
            pass
        # Extract description
        try:
            description_elem = soup.find('div', class_='div_textqc').find('small', class_='text_qc')
            description = description_elem.text.strip() if description_elem else ''
        except:
            pass
        # Extract email and website
        try:
            email_web_div = soup.find('div', class_='email_web_section')
            if email_web_div:
                # Find the <a> elements within the div
                email_web_links = email_web_div.find_all('a')
                for link in email_web_links:
                    href = link.get('href', '').strip()
                    if href.startswith('mailto:'):
                        email = href.replace('mailto:', '')
                    elif href and not href.startswith('javascript:'):
                        website = href
        except:
            pass
        return [nganh_listing_txt, logo_url, company_name, description, address, district, email, website, is_verified, ', '.join(phone_numbers)]
    except Exception as e:
        print(e)
        return None


In [67]:
csv_file_path = 'output.csv'  # Replace with your desired CSV file path

In [208]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

excel_file_path = 'output.xlsx'

# Function to extract page IDs and click each page to crawl data
def crawl_multiple_pages(url, delay=20):
    options = Options()
    options.page_load_strategy = 'normal'
    driver = webdriver.Chrome(options=options)

    driver.get(url)
    driver.implicitly_wait(delay)

    try:
        # Extract the content inside the iframe
        iframe_content = driver.page_source

        # Parse the iframe content with BeautifulSoup
        soup = BeautifulSoup(iframe_content, 'html.parser')

        # Extract and process data as needed
        div_list_company = soup.find_all('div', class_='div_list_cty')
        div_list_company_element = div_list_company[0].find_all('div', class_='w-100 h-auto shadow rounded-3 bg-white p-2 mb-3')
        print(len(div_list_company_element))
        list_data = [] 
        for div_company in div_list_company_element:
            data = extractInfo(div_company)
            if data:
                append_to_excel(data, 'oc-vit-bulong.xlsx')
                print(data)
        
    except Exception as e:
        print(e)
    finally:
        # Close the browser
        driver.quit()

# Example usage:
crawl_multiple_pages('https://trangvangvietnam.com/cateprovinces/31960/%E1%BB%91c-v%C3%ADt-bulong-bulong-%E1%BB%91c-v%C3%ADt-%E1%BB%9F-t%E1%BA%A1i-tp.-h%E1%BB%93-ch%C3%AD-minh-(tphcm).html?page=6')


32
['ốc vít bulong bulong ốc vít  ở tại  tp. hồ chí minh', '', 'EJOT BUIDING FASTENERS REP. OFFICE', '', '173A Nguyễn Văn Trỗi, P. 11, Q. Phú Nhuận, Tp. Hồ Chí Minh, Việt Nam', '', '', '#', 'no', '']
['ốc vít bulong bulong ốc vít  ở tại  tp. hồ chí minh', '', 'Công Ty TNHH CNF Hardware', '', 'Số 333, Đường Zhen Jia Dai , Haiyan, Jiaxing Tp. Hồ Chí Minh,', '', '', '', 'no', '']
['ốc vít bulong bulong ốc vít  ở tại  tp. hồ chí minh', '', 'Công Ty Cổ Phần Bulông & Cáp Thép Sơn Hà', '', '508 C/c An Lộc 2, Đường Số 3, KĐTM An Phú An Khánh, P. An Phú, Q. 2, Tp. Hồ Chí Minh, Việt Nam', '', '', '#', 'no', '']
['ốc vít bulong bulong ốc vít  ở tại  tp. hồ chí minh', '', 'Công Ty TNHH Đầu Tư Thương Mại Dịch Vụ Tân Vinh Lộc', '', '227/24 Nguyễn Thị Nhỏ, P. 16, Q. 11, Tp. Hồ Chí Minh, Việt Nam', '', '', '', 'no', '']
['ốc vít bulong bulong ốc vít  ở tại  tp. hồ chí minh', '', 'Công Ty TNHH MTV TM DV Mai Tech', '', 'Số 80 Calmet, P. Nguyễn Thái Bình, Q. 1, Tp. Hồ Chí Minh, Việt Nam', '', '', '#', 'n

In [198]:
import openpyxl

def append_to_excel(data, excel_file):
    try:
        # Load the existing workbook
        workbook = openpyxl.load_workbook(excel_file)
    except FileNotFoundError:
        # If the file doesn't exist, create a new workbook
        workbook = openpyxl.Workbook()

    # Select the active sheet (usually the first sheet)
    sheet = workbook.active

    # Find the next available row in the sheet
    next_row = sheet.max_row + 1

    # Iterate through the data and write to the next available row
    for col, value in enumerate(data, start=1):
        cell = sheet.cell(row=next_row, column=col)
        cell.value = value

    # Save the workbook to the Excel file, overwriting the existing file
    workbook.save(excel_file)

# # Example usage:
# data = [
#     'bàn ghế nội thất  ở tại  tp. hồ chí minh', '', 'Công Ty TNHH Thiết Kế Thi Công Nội Thất Tinhome', '',
#     'Số 6, Đường Số 8, P. Bình Hưng Hòa B, Q. Bình Tân, Tp. Hồ Chí Minh, Việt Nam', 'Q. Bình Tân',
#     'tinhomevn@gmail.com', 'http://tinhome.vn', 'no', '0908391297'
# ]

In [149]:


# Specify the Excel file path

# Specify the Excel sheet name (optional)
# sheet_name = 'Sheet1'

# Call the function to write data to the Excel file

In [147]:
pip install openpyxl

9563.96s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached openpyxl-3.1.2-py2.py3-none-any.whl (249 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
You should consider upgrading via the '/mnt/mydisk/.pyenv/versions/3.8.16/envs/crawl-data-for-sale/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
